In [ ]:
#机器之心
#lenet5
#卷积1
#平均池化1
#卷积2
#平均池化2
#全连1
#全连2
#softmax
#激活为 sigmoid
'''
28 28 1
卷积1 28 28 6
池化1 14 14 6
卷积2（padding='VALID'） 10 10 16
池化2 5 5 16
hidden1 120
hidden2 84
output 10
'''

In [1]:
import numpy as np
import tensorflow as tf
import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/' , one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [38]:
def weights_variable(shape):
    initial = tf.truncated_normal(shape=shape , stddev=0.1)
    return tf.Variable(initial)

def biases_variable(shape):
    initial = tf.constant(0.1 , shape=shape)
    return tf.Variable(initial)

In [58]:
xs_ori = tf.placeholder(tf.float32 , shape = [None , 784])

xs = tf.reshape(xs_ori , shape=[-1 , 28 , 28 , 1])
ys = tf.placeholder(tf.float32 , shape = [None , 10])

#卷积1
w1 = weights_variable([5 , 5 , 1 , 32])
b1 = biases_variable([32])
layer1_conv = tf.nn.conv2d(xs , w1 , strides=[1,1,1,1] , padding='SAME')
layer1_actv = tf.nn.relu(layer1_conv + b1)
layer1_pool = tf.nn.avg_pool(layer1_actv , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

#卷积2
w2 = weights_variable([5 , 5 , 32 , 64])
b2 = biases_variable([64])
layer2_conv = tf.nn.conv2d(layer1_pool , w2 , strides=[1,1,1,1] , padding='SAME')
layer2_actv = tf.nn.relu(layer2_conv + b2)
layer2_pool = tf.nn.avg_pool(layer2_actv , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

#展平
flat_layer = tf.reshape(layer2_pool , shape = [-1 , 7*7*64])

#全连接1
#图片经conv1 pool1 conv2 pool2之后 变为5*5 channel=patch_depth2
w3 = weights_variable([7*7*64 , 1024])
b3 = biases_variable([1024])
layer3_fccd = tf.matmul(flat_layer , w3)+b3
layer3_actv = tf.nn.relu(layer3_fccd)

#全连接2
w4 = weights_variable([1024 , 100])
b4 = biases_variable([100])
layer4_fccd = tf.matmul(layer3_actv , w4) + b4
layer4_actv = tf.nn.relu(layer4_fccd)

#output layer
w5 = weights_variable([100 , 10])
b5 = biases_variable([10])
pred = tf.nn.softmax(tf.matmul(layer4_actv , w5) + b5)


loss = -tf.reduce_sum(ys * tf.log(pred))

train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

valid_feed = {xs_ori:mnist.validation.images , ys:mnist.validation.labels}

#====
right = tf.equal(tf.argmax(pred , 1) , tf.argmax(ys , 1))
accuracy = tf.reduce_mean(tf.cast(right , tf.float32))

for i in range(30):
    train_x , train_y = mnist.train.next_batch(100)
    sess.run(train_step , feed_dict = {xs_ori:train_x , ys:train_y})
    
    print(sess.run(loss , feed_dict = {xs_ori:train_x , ys:train_y}))
    
    if i % 100 == 0:
        print(sess.run(accuracy , feed_dict=valid_feed))

944.656
0.0764
394.022
235.274
224.477
216.381
214.275
224.89
221.136
210.128
194.935
193.309
173.94
160.655
151.367
202.111
243.47
176.65
145.387
122.271
139.6
229.536
171.838
158.289
117.119
138.111
111.163
80.2798
70.4362
69.8879
70.7816


In [59]:
test_feed = {xs_ori:mnist.test.images , ys:mnist.test.labels}

print(sess.run(accuracy , feed_dict=test_feed))

0.7229
